In [1]:
# Boilerplate
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from matplotlib import animation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
import time
import os
import warnings
from re import match, search
from random import randint
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import ipywidgets as wgt
import dill
import tmv3 as tm
from tmv3 import Lt_from_hc_GammaChi  # Necessary when loading previously-saved function objects

# Import changes from editing TableMaker
import importlib
importlib.reload(tm)

assert callable(Lt_from_hc_GammaChi), "Lt_from_hc_GammaChi must be callable"

In [ ]:
import tmv3_class as tmv3c
importlib.reload(tmv3c)

<module 'tmv3_class' from 'c:\\Users\\jared\\Desktop\\TableMaker\\tmv3_class.py'>

In [ ]:
##### Set parameters for the table
numXim = 150
ximLfrac = 0.2
ximGfrac = 0.5
numXiv = 30

# Set filepath
path = r'./data/ChiGammaTablev3'
file_pattern = r'flm_.*.dat$'

# Adjust data parameters (change depending on how ignis was run)
tvals = np.arange(0,14,1)
Lvals = np.arange(0,26,1)
gammaValues = [0., 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65]
Lbounds = [min(Lvals), max(Lvals)]
tbounds = [min(tvals), max(tvals)]
path_to_hsens = './data/ChiGammaTablev3/hsens.dat'

In [ ]:
# Class for the table
tableDemo = tmv3c.table(path, Lvals, tvals, nxim=5, nxiv=5, ximLfrac=ximLfrac, ximGfrac=ximGfrac,
                        gammaValues=gammaValues, flmt_file_pattern=r'^flm.*.dat$')

In [ ]:
# Create a temperature table
tableDemo.phi_mvhc('T')

Beginning parallel table creation...


({'h': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>,
  'c': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>,
  'T': <function tmv3_class.table.phi_mvhc.<locals>.create_phi_table.<locals>.phi_table(xim, xiv, h, c, useStoredSolution=True, solver='gammachi', detailedWarn=False, maxIter=100, saveSolverStates=False, LstepParams=[0.25, 0.01, 0.003], tstepParams=[0.25, 9.5, 0.02])>},
 {})

In [ ]:
result_demo = tm.phi_mvhc(path, Lvals, tvals, file_pattern = file_pattern, phi = ['T'], 
                        numXim = 5, numXiv = 5, parallel = True,
                        ximLfrac = ximLfrac, ximGfrac = ximGfrac)

funcs_demo, tableInfo_demo = result_demo
Tfunc_demo = funcs_demo[0]
h_table_demo = tableInfo_demo[0][0]
h_indices_demo = tableInfo_demo[0][1]
c_table_demo = tableInfo_demo[1][0]
c_indices_demo = tableInfo_demo[1][1]
T_table_demo, T_indices_demo = tableInfo_demo[2][0], tableInfo_demo[2][1]

Completed data import ('get_data_files')
Beginning parallel table creation...


In [ ]:
from scipy.optimize import differential_evolution
import numpy as np

h = 0.0
c = 0.0

def neg_abs_diff(mv):
    xim, xivRel = mv
    xim = np.clip(xim, 0.0, 1.0)
    xivRel = np.clip(xivRel, 0.0, 1.0)
    xiv = xim * (1 - xim) * xivRel

    T1 = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
    T2 = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)

    return -abs(T1 - T2)

bounds = [(0.0, 1.0), (0.0, 1.0)]

result = differential_evolution(neg_abs_diff, bounds, seed=0, 
                                maxiter=10, tol=1e-2)

print("Max absolute difference:", -result.fun)
print("Location of max difference:", result.x)

Max absolute difference: 0.09688861076210742
Location of max difference: [0.403026   0.21280863]


In [ ]:
# Validate DE's solution
xim, xivRel = result.x
xiv = xim * (1 - xim) * xivRel
T1 = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
T2 = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)
print(f"T1 = {T1}, T2 = {T2}, Difference = {abs(T1 - T2)}")

T1 = [1643.4193789], T2 = [1643.4193789], Difference = [0.]


In [ ]:
ximArr = np.arange(0, 1, 97)
xivArr = np.linspace(0, 1, 98)
h = 0.0
c = 0.0
for i in range(len(ximArr)):
    for j in range(len(xivArr)):
        xim = ximArr[i]
        xiv = xivArr[j]*xim*(1 - xim)

        classVal = tableDemo.phi_mvhc_funcs['T'](xim, xiv, h, c)
        pakijVal = Tfunc_demo(xim, xiv, h, c, gammaValues=gammaValues, path_to_hsens=path_to_hsens)
        
        if not np.isclose(classVal, pakijVal):
            print(f"Discrepancy found at xim={xim}, xiv={xiv}: classVal-pakijVal={classVal-pakijVal}")
            print()